In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import openpyxl
from openpyxl.styles import Font
import time

In [ ]:
#NOTE async, await로 셀레니움 끊기는 것을 막을 수 있을까?

In [25]:
class ExplorerCrawler:
    def __init__(self, workbook, sheet):
        self.color = Font(name="Arial", color="8f34eb")
        self.workbook = openpyxl.load_workbook(filename=workbook, data_only=True)
        self.sheet = sheet

        self.options = webdriver.ChromeOptions()
        self.options.add_argument("headless")
        self.options.add_argument("--window-size=1920,1080")
        self.options.add_argument(f'user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36')
        self.only_address_map = {
            'ETH': [ ['ETH', "ETC"], "https://etherscan.io/tx/", 
            "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]", True],
            'BSC': [ ['BSC'], "https://bscscan.com/tx/", 
            "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]", True],
            'TRX': [ ["TRX", "TRON", "TRC20USDT"], "https://tronscan.org/#/transaction/", 
            "//*[@id='root']/div[2]/main/div/div/div[3]/div[2]/div/div/div/table/tbody/tr[1]/td/span/div/span/div/div/span/div/a/div",
            False],
            "HECO": [ ["HECO", "HRC"], "https://hecoinfo.com/tx/", 
            "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]", True ],
            "MATIC": [ ["MATIC"], "https://polygonscan.com/tx/", 
            "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]", True ],
            "ETC": [ ["ETC"], "https://etcblockexplorer.com/tx/", 
            "//*[@id='wrap']/div/div[3]/div/div[2]/div[1]/div/table/tbody/tr/td/span/a", True ],
            "KLAY": [ ["KLAY"], "https://scope.klaytn.com/tx/", 
            "//*[@id='root']/div/div[2]/div[2]/div/div/div/div/div/div[1]/div[3]/div[2]/div/span[1]/div/a", True],
            "BTC": [ ["BTC"], "https://blockstream.info/tx/", 
            "//*[@id='transaction-box']/div[2]/div[3]", False]
        }
        self.with_memo_map = {
            "ATOM": [ ["ATOM"], "https://atomscan.com/transactions/", 
            "//*[@id='app']/div[2]/div/div/div[2]/div/div[3]/div/article/div[2]/div[1]/div[2]/span/span/a/span",
            "//*[@id='app']/div[2]/div/div/div[2]/div/div[2]/div/div/div/div[6]/div[2]",
            False ],
            "EOS": [ ["EOS"], "https://eosflare.io/tx/", 
            "//*[@id='actions']/div[1]/div[3]/div/div[1]/a[1]",
            "//*[@id='actions']/div[1]/div[3]/div/div[2]",
            False ],
            "TERRA": [ ["TERRA"], "https://finder.terra.money/classic/tx/", 
            "//*[@id='root']/section/section/div[3]/div[2]/section[3]/div/a",
            "//*[@id='root']/section/section/div[2]/div[6]/div[2]",
            False ],
            "XRP": [ ["XRP"], "https://xrpscan.com/tx/",
            "//*[@id='root']/div/div/div/div/div[3]/div/div[1]/div[1]/div[2]/div[1]/div/table/tbody/tr[3]/td[2]/a",
            "//*[@id='root']/div/div/div/div/div[3]/div/div[1]/div[1]/div[2]/div[1]/div/table/tbody/tr[4]/td[2]/span/span/span",
            False ],
            "XLM": [ ["XLM"], "https://steexp.com/tx/",
            "//*[@id='operation-table']/tbody/tr/td[1]/span/a",
            "//*[@id='main-content']/div/div[1]/div/div[2]/table/tbody/tr[4]/td[2]",
            False ]
        }
        
    def _get_webdriver(self, options):
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)    
        return driver

    def logger(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            func(*args, **kwargs)
            print(f"took {time.time() - start_time} seconds")
        return wrapper
    
    # -------------------------------------------------------------------------------------------------------------------------

    @logger
    def get_ETH_address(self, start, target_network = "ETH"):
        driver = self._get_webdriver(self.options)
        workbook = self.workbook
        sheet = workbook[self.sheet]
        for i in range(start, sheet.max_row + 1):
            network = sheet[f"L{str(i)}"].value
            if network in self.only_address_map[target_network][0]:
                tx_hash = sheet[f"S{str(i)}"].value
                if "Internal" in tx_hash or sheet[f"N{str(i)}"].value is not None: 
                    continue
                
                if self.only_address_map[target_network][3]:
                    tx_hash = tx_hash if tx_hash[:2] == "0x" else "0x" + tx_hash
                else:
                    pass
                driver.get(  self.only_address_map[target_network][1] + tx_hash )
                driver.implicitly_wait(10)
                try:
                    element = driver.find_element(by=By.XPATH, value= self.only_address_map[target_network][2])
                    driver.implicitly_wait(10)   
                    from_address = element.text
                    sheet[f"N{str(i)}"].value = from_address
                    print("row:", i, "from_address:", from_address, "txhash:", tx_hash)
                except:
                    print("Wrong chain or invalid existing tx_hash")
                sheet[f"N{str(i)}"].font = self.color
        driver.quit()
        workbook.save(f"files/{target_network} 완료.xlsx")

    @logger
    def get_BSC_address(self, start, target_network = "BSC"):
        self.get_ETH_address(start, target_network)

    @logger
    def get_HECO_address(self, start, target_network = "HECO"):
        self.get_ETH_address(start, target_network)

    @logger
    def get_MATIC_address(self, start, target_network = "ETC"):
        self.get_ETH_address(start, target_network)
    
    @logger
    def get_ETC_address(self, start, target_network = "TRX"):
        self.get_ETH_address(start, target_network)


    @logger
    def get_TRX_address(self, start, target_network = "TRX"):
        # owner 인지, from인지에 따라 달라서 여러번에 걸쳐 했었음
        # 나중에 하자
        pass

    @logger
    def get_BTC_address(self, start, target_network = "BTC"):
        
        def _get_matched_from_address(QUANTITY, result):
            for i in range(len(result)):
                if i % 2 == 1:
                    BTC_AMOUNT = result[i]
                    AMOUNT, _ = BTC_AMOUNT.split()
                    AMOUNT = float(AMOUNT)

                    if round(QUANTITY, 3) == round(AMOUNT, 3):
                        return result[i-1]

        driver = self._get_webdriver(self.options)
        workbook = self.workbook
        sheet = workbook[self.sheet]
        for i in range(start, sheet.max_row + 1):
            network = sheet[f"L{str(i)}"].value
            if network in self.only_address_map[target_network][0]:
                tx_hash = sheet[f"S{str(i)}"].value
                if "Internal" in tx_hash or sheet[f"N{str(i)}"].value is not None: 
                    continue
                
                if self.only_address_map[target_network][3]:
                    tx_hash = tx_hash if tx_hash[:2] == "0x" else "0x" + tx_hash
                else:
                    pass
                driver.implicitly_wait(10)
                driver.get(  self.only_address_map[target_network][1] + tx_hash )
                driver.implicitly_wait(10)
                try:
                    element = driver.find_element(by=By.XPATH, value= self.only_address_map[target_network][2])
                    driver.implicitly_wait(10) 

                    QUANTITY = sheet[f"M{str(i)}"].value
                    result = element.text.split("\n")
                    from_address = _get_matched_from_address(QUANTITY, result)

                    sheet[f"N{str(i)}"].value = from_address
                    print("row:", i, "from_address:", from_address, "txhash:", tx_hash)
                except:
                    print(f"row: {i}, wrong chain or invalid existing tx_hash")
                sheet[f"N{str(i)}"].font = self.color
        driver.quit()
        workbook.save(f"files/{target_network} 완료.xlsx")
    

    @logger
    def get_KLAY_address(self, start, target_network = "KLAY"):
        driver = self._get_webdriver(self.options)
        workbook = self.workbook
        sheet = workbook[self.sheet]
        
        for i in range(start, sheet.max_row + 1):
            network = sheet[f"L{str(i)}"].value
            if network in self.only_address_map[target_network][0]:
                tx_hash = sheet[f"S{str(i)}"].value

                if "Internal" in tx_hash or sheet[f"N{str(i)}"].value is not None: 
                    continue
                
                if self.only_address_map[target_network][3]:
                    tx_hash = tx_hash if tx_hash[:2] == "0x" else "0x" + tx_hash
                else:
                    pass
                
                driver.get(  self.only_address_map[target_network][1] + tx_hash )
                driver.implicitly_wait(10)
                try:
                    element = driver.find_element(by=By.XPATH, value= self.only_address_map[target_network][2])
                    driver.implicitly_wait(10)   
                    address = element.get_attribute('href').split("/")[-1]
                    exchange = element.text
                    from_address = address + " (" + exchange + ")"
                    sheet[f"N{str(i)}"].value = from_address 
                    print("row:", i, "from_address:", from_address, "txhash:", tx_hash)
                except:
                    print("Wrong chain or invalid existing tx_hash")
                sheet[f"N{str(i)}"].font = self.color
        driver.quit()
        workbook.save(f"files/{target_network} 완료.xlsx")

    # ------------------------------------------------------------------------------------------------------------
    @logger
    def get_ATOM_address_memo(self, start, target_network = "ATOM"):
        driver = self._get_webdriver(self.options)
        workbook = self.workbook
        sheet = workbook[self.sheet]

        for i in range(start, sheet.max_row + 1):
            network = sheet[f"L{str(i)}"].value
            if network in self.with_memo_map[target_network][0]:
                tx_hash = sheet[f"S{str(i)}"].value
                if "Internal" in tx_hash or sheet[f"N{str(i)}"].value is not None: #요게 잘못됨, 무조건 continue하게 되어있음
                    continue
                # invalid할 때

                if self.with_memo_map[target_network][4]:
                    tx_hash = tx_hash if tx_hash[:2] == "0x" else "0x" + tx_hash
                else:
                    pass
                # driver = webdriver.Chrome("chromedriver.exe", options=options) 
                driver.get(  self.with_memo_map[target_network][1] + tx_hash)
                driver.implicitly_wait(10)
                try:
                    from_address_element = driver.find_element(by=By.XPATH, value=self.with_memo_map[target_network][2])
                    driver.implicitly_wait(10)
                    from_address = from_address_element.get_attribute("href").split("/")[-1]
                    sheet[f"N{str(i)}"].value = from_address
                    memo_element = driver.find_element(by=By.XPATH, value=self.with_memo_map[target_network][3])
                    driver.implicitly_wait(10)
                    memo = memo_element.text
                    print("row:", i, "from_address:", from_address, "memo:", memo, "txhash:", tx_hash)
                    sheet[f"O{str(i)}"].value = memo

                    num_written += 1    
                except:
                    print("Wrong chain or invalid existing tx_hash")

                sheet[f"N{str(i)}"].font = self.color
                sheet[f"O{str(i)}"].font = self.color
                driver.implicitly_wait(5)

            driver.quit()

    @logger
    def get_EOS_address_memo(self, start, target_network = "EOS"):
        pass

    @logger
    def get_TERRA_address_memo(self, start, target_network = "TERRA"):
        pass
    
    @logger
    def get_XRP_address_memo(self, start, target_network = "XRP"):
        from_address_href = from_address_element.get_attribute('href').split('/')[-1]
        from_address_description = from_address_element.text
        from_address = from_address_href + " (" + from_address_description+ ")"
        memo = memo_element.text
        memo = memo.split(":")[-1].strip()
        pass

    @logger
    def get_XLM_address_memo(self, start, target_network = "XLM"):

        from_address = former.get_attribute("href").split("/")[-1]
        memo = latter.text
        pass


In [26]:
filename = "files/XLM 완료 Bbrick_취합파일 정리_2200617(원태님).xlsx"
ec = ExplorerCrawler(filename, "입금")
ec.get_BTC_address(1)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


row: 47 from_address: 3LLRS688CkBg71m4qvEffbHh9sMQZfH5or txhash: 92bc4e3ced963c5f2a6588e77c29cdff391e798932228fa5773bd431edd63454
row: 75 from_address: 3LLRS688CkBg71m4qvEffbHh9sMQZfH5or txhash: 723f086da53f3f1a94b7ea7c8479bdbf5a1390a38d55aa946fb3fbb194577d7f
row: 76 from_address: 3LLRS688CkBg71m4qvEffbHh9sMQZfH5or txhash: 1eed82b9ca8c945bc9ce3938ffaca5481471626ebf417d82ca89be7407715e5c
row: 77 from_address: 3LLRS688CkBg71m4qvEffbHh9sMQZfH5or txhash: 05a36a0959b4bd514a314dbbdbef47f2758a625051660ab768ed4be37e97c455
row: 81 from_address: 3LLRS688CkBg71m4qvEffbHh9sMQZfH5or txhash: 7e6d2e6ed45c08721706dad8badf1e08c708a5e50f094862e3a4e550b9562497
row: 82 from_address: 3LLRS688CkBg71m4qvEffbHh9sMQZfH5or txhash: 76837738229516e036193677e9ba5767b23ca00a5c57c9dc91957ccbfc44d28c
row: 87, wrong chain or invalid existing tx_hash
row: 92 from_address: 3LLRS688CkBg71m4qvEffbHh9sMQZfH5or txhash: 5b1cca3953c1f5c679f159b83d80110b5c2313d1b555f442f8b67da638582f98
row: 93 from_address: 3LLRS688CkBg71m4qvE